This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '---'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import collections


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/metadata
# https://www.quandl.com/api/v3/datasets/WIKI/FB.json?column_index=4&start_date=2014-01-01&end_date=2014-12-31&collapse=monthly&transform=rdiff&api_key=YOURAPIKEY

r = requests.get("https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?start_date=2014-01-01&end_date=2014-01-02&api_key=" + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()['dataset_data']

{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Volume',
  'Ex-Dividend',
  'Split Ratio',
  'Adj. Open',
  'Adj. High',
  'Adj. Low',
  'Adj. Close',
  'Adj. Volume'],
 'start_date': '2014-01-01',
 'end_date': '2014-01-02',
 'frequency': 'daily',
 'data': [['2014-01-02',
   54.83,
   55.22,
   54.19,
   54.71,
   43195500.0,
   0.0,
   1.0,
   54.83,
   55.22,
   54.19,
   54.71,
   43195500.0]],
 'collapse': None,
 'order': None}

After going into the dataset_data, we can find a few keys that are interesting. 'Column_names' is probably good to know and they probably refer to the 'data'. The 'data' is an array of lists that correspont to the 'Column_names'. For example, if I want to get the 'High' for a particular day (let's say the first day of the dataset), I would just have to call data[0][2].

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Request data from the Frankfurt Stock Exchnage for the ticker AFX_X for the year 2017

In [5]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)

### Convert the returned JSON object into a python dictionary

In [9]:
r_dict = r.json()
print(type(r_dict))

r_data = r_dict['dataset']['data']

<class 'dict'>


### Calculate what the highest and lowest opening prices were for the stock in this period.

In [10]:
highest_opening_price = 0
lowest_opening_price = 100000000

for data in r_data:
    try: # there's a NoneType which comparators don't like
        if data[1] > highest_opening_price:
            highest_opening_price = data[1]    
        if data[1] < lowest_opening_price:
            lowest_opening_price = data[1]
    
    except:
        pass

print('The highest opening price is: ' + str(highest_opening_price))
print('The lowest opening price is: ' + str(lowest_opening_price))

The highest opening price is: 53.11
The lowest opening price is: 34.0


In [11]:
# doing this better:

opening_price = [data[1] for data in r_data if data[1] is not None]
print('The highest opening price is: ' + str(max(opening_price)))
print('The lowest opening price is: ' + str(min(opening_price)))

The highest opening price is: 53.11
The lowest opening price is: 34.0


### What was the largest change in any one day (based on High and Low price)?

In [12]:
price_change = [data[2] - data[3] for data in r_data if ((data[2] is not None) and (data[3] is not None))]
print('The largest change in one day is: ' + str(max(price_change)))

The largest change in one day is: 2.8100000000000023


### What was the largest change between any two days (based on Closing Price)?

In [13]:
closing_price = [data[4] for data in r_data]

largest_change = 0
prev_close = None
change = None
change_list = []

for close in closing_price:
    try:
        change = close - prev_close
        change_list.append(change)
        
        if abs(change) > abs(largest_change):
            largest_change = change
        
    except:
        pass
    
    prev_close = close

print('The largest change between any two days is: ' + str(largest_change))

The largest change between any two days is: 2.559999999999995


### What was the average daily trading volume during this year?

In [14]:
trading_volume = [data[6] for data in r_data if data[6] is not None]
trading_volume_avg = sum(trading_volume)/len(trading_volume)

print('The average daily trading volume during the year is: ' + str(trading_volume_avg))

The average daily trading volume during the year is: 89124.33725490196


### (Optional) What was the median trading volume during this year. 
(Note: you may need to implement your own function for calculating the median.)


In [15]:
trading_volume.sort()
median = trading_volume[int(len(trading_volume)/2+1)] # len() is odd so this is the middle of data

print('The median trading volume during the year is: ' + str(median))

The median trading volume during the year is: 76600.0
